In [8]:
import os
import fitz
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import spacy
from sentence_transformers import SentenceTransformer, util
import torch

pd.set_option('display.max_colwidth', None)
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

def tokenize(text):
    doc = nlp(text.lower())
    tokens = set()
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.like_num and token.is_alpha:
            tokens.add(token.lemma_)
    return tokens

def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if union else 0

def process_resumes(folder_path):
    resume_data = {}
    for fname in os.listdir(folder_path):
        if fname.endswith('.pdf'):
            full_path = os.path.join(folder_path, fname)
            text = extract_text_from_pdf(full_path)
            tokens = tokenize(text)
            resume_data[fname] = tokens
    return resume_data

def read_resumes_raw(base_dir, categories):
    texts = []
    names = []
    for category in categories:
        folder = os.path.join(base_dir, category)
        for fname in os.listdir(folder):
            if fname.endswith(".pdf"):
                full_path = os.path.join(folder, fname)
                text = extract_text_from_pdf(full_path)
                texts.append(text)
                names.append(fname)
    return names, texts

def process_job_descriptions(csv_path):
    df = pd.read_csv(csv_path)
    jd_data = {}
    for idx, row in df.iterrows():
        jd_text = str(row['Job Description'])
        tokens = tokenize(jd_text)
        jd_data[idx] = {
            'job_title': row.get('Job Title', ''),
            'tokens': tokens,
            'full_desc': jd_text.strip()
        }
    return jd_data

def read_job_descriptions(csv_path):
    df = pd.read_csv(csv_path)
    jd_texts = df['Job Description'].astype(str).tolist()
    jd_titles = df['Job Title'].astype(str).tolist()
    return jd_titles, jd_texts

def match_resumes_to_jobs(resume_tokens, jd_data, top_k=10):
    results = defaultdict(list)
    for resume_name, r_tokens in resume_tokens.items():
        scores = []
        for jd_id, jd_info in jd_data.items():
            score = jaccard_similarity(r_tokens, jd_info['tokens'])
            scores.append((jd_id, score, jd_info['job_title']))
        top_matches = sorted(scores, key=lambda x: -x[1])[:top_k]
        results[resume_name] = top_matches
    return results

def match_resumes_tfidf_cosine(resume_names, resume_texts, jd_titles, jd_texts, top_k=10):
    all_docs = resume_texts + jd_texts
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(all_docs)

    resume_vecs = tfidf_matrix[:len(resume_texts)]
    jd_vecs = tfidf_matrix[len(resume_texts):]

    sim_matrix = cosine_similarity(resume_vecs, jd_vecs)

    results = {}
    for i, resume_name in enumerate(resume_names):
        top_matches = sorted(
            list(enumerate(sim_matrix[i])), key=lambda x: -x[1]
        )[:top_k]
        results[resume_name] = [(idx, score, jd_titles[idx]) for idx, score in top_matches]
    return results, vectorizer, jd_texts

def match_resumes_sbert(resume_names, resume_texts, jd_titles, jd_texts, top_k=10):
    # model = SentenceTransformer('all-MiniLM-L6-v2')
    model = SentenceTransformer('all-mpnet-base-v2')
    resume_embeddings = model.encode(resume_texts, convert_to_tensor=True, show_progress_bar=True)
    jd_embeddings = model.encode(jd_texts, convert_to_tensor=True, show_progress_bar=True)

    results = {}
    for i, resume_name in enumerate(resume_names):
        cosine_scores = util.cos_sim(resume_embeddings[i], jd_embeddings)[0]
        top_results = torch.topk(cosine_scores, k=top_k)
        top_matches = [(int(idx), float(cosine_scores[idx]), jd_titles[int(idx)]) for idx in top_results.indices]
        results[resume_name] = top_matches
    return results, model

def compare_topk_overlap(resume_name, jaccard_results, tfidf_results):
    jaccard_ids = {jd_id for jd_id, _, _ in jaccard_results.get(resume_name, [])}
    tfidf_ids = {jd_id for jd_id, _, _ in tfidf_results.get(resume_name, [])}

    overlap = jaccard_ids.intersection(tfidf_ids)

    print(f"\nResume: {resume_name}")
    print(f"Jaccard Top-K IDs: {sorted(jaccard_ids)}")
    print(f"TF-IDF Top-K IDs: {sorted(tfidf_ids)}")
    print(f"Overlap ({len(overlap)}): {sorted(overlap)}")
    print(f"Jaccard-Only: {sorted(jaccard_ids - tfidf_ids)}")
    print(f"TF-IDF-Only: {sorted(tfidf_ids - jaccard_ids)}")

def plot_score_distribution(jaccard_results, tfidf_results):
    jaccard_scores = [score for matches in jaccard_results.values() for _, score, _ in matches]
    tfidf_scores = [score for matches in tfidf_results.values() for _, score, _ in matches]

    plt.figure(figsize=(10, 5))
    plt.hist(jaccard_scores, bins=20, alpha=0.6, label='Jaccard', color='skyblue')
    plt.hist(tfidf_scores, bins=20, alpha=0.6, label='TF-IDF', color='salmon')
    plt.title("Similarity Score Distribution: Jaccard vs TF-IDF")
    plt.xlabel("Similarity Score")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [9]:
categories = ['INFORMATION-TECHNOLOGY']
base_resume_dir = './resume_pds/data/data'
jd_csv_path = 'job_title_des.csv'

resume_tokens = {}
for category in categories:
    folder_path = os.path.join(base_resume_dir, category)
    resume_tokens.update(process_resumes(folder_path))

resume_names, resume_texts = read_resumes_raw(base_resume_dir, categories)
jd_titles, jd_texts = read_job_descriptions(jd_csv_path)

jd_data = process_job_descriptions(jd_csv_path)

In [10]:
# results_jaccard = match_resumes_to_jobs(resume_tokens, jd_data)
# results_tfidf, vectorizer, all_jd_texts = match_resumes_tfidf_cosine(resume_names, resume_texts, jd_titles, jd_texts)

In [11]:
results_sbert, sbert_model = match_resumes_sbert(resume_names, resume_texts, jd_titles, jd_texts)

Batches: 100%|██████████████████████████████████| 72/72 [00:54<00:00,  1.33it/s]


In [12]:
# Print top matches using SBERT for a specific resume
target_resume = "20674668.pdf"
print(f"\nTop SBERT matches for {target_resume}:")
for jd_id, score, title in results_sbert.get(target_resume, []):
    print(f"  JD ID: {jd_id} | Title: {title[:40]}... | SBERT Score: {score:.3f}")


Top SBERT matches for 20674668.pdf:
  JD ID: 128 | Title: JavaScript Developer... | SBERT Score: 0.781
  JD ID: 940 | Title: JavaScript Developer... | SBERT Score: 0.771
  JD ID: 1790 | Title: Java Developer... | SBERT Score: 0.762
  JD ID: 1894 | Title: Backend Developer... | SBERT Score: 0.759
  JD ID: 1466 | Title: Software Engineer... | SBERT Score: 0.758
  JD ID: 487 | Title: Wordpress Developer... | SBERT Score: 0.754
  JD ID: 1772 | Title: Full Stack Developer... | SBERT Score: 0.752
  JD ID: 1425 | Title: PHP Developer... | SBERT Score: 0.750
  JD ID: 2185 | Title: Wordpress Developer... | SBERT Score: 0.749
  JD ID: 1519 | Title: Django Developer... | SBERT Score: 0.747


In [13]:
import pandas as pd
import math

# 1) Load your gold‐standard CSVs
res_df = pd.read_csv('gold_resumes.csv')  # cols: Resume_ID, Resume_Text
jd_df  = pd.read_csv('gold_jds.csv')      # cols: Job_ID,    Job_Text

# 2) Prepare the inputs for SBERT matcher
resume_names = res_df['Resume_ID'].astype(str).tolist()
resume_texts = res_df['Resume_Text'].astype(str).tolist()
jd_ids       = jd_df['Job_ID'].astype(str).tolist()
jd_texts     = jd_df['Job_Text'].astype(str).tolist()

# 3) Run your SBERT matcher on the gold data (top_k=4)
results_sbert, sbert_model = match_resumes_sbert(
    resume_names, resume_texts,
    jd_ids, jd_texts,
    top_k=4
)

Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  2.24it/s]


In [14]:
gold = {
    "R1": ["J1","J2","J3","J4"],
    "R2": ["J5","J6","J7","J8"],
    "R3": ["J9","J10","J11","J12"],
    "R4": ["J13","J14","J15","J16"],
    "R5": ["J17","J18","J19","J20"],
}

In [15]:
def precision_at_k(recs, gold_set, k):
    return len(set(recs[:k]) & gold_set) / k

def recall_at_k(recs, gold_set, k):
    return len(set(recs[:k]) & gold_set) / len(gold_set)

def topk_accuracy(recs, gold_set, k):
    return 1.0 if set(recs[:k]) & gold_set else 0.0

def reciprocal_rank(recs, gold_set):
    for i, jid in enumerate(recs, start=1):
        if jid in gold_set:
            return 1.0 / i
    return 0.0

def dcg_at_k(rels, k):
    return sum(r / math.log2(idx+1) for idx, r in enumerate(rels[:k], start=1))

def ndcg_at_k(recs, gold_set, k):
    rels  = [1 if jid in gold_set else 0 for jid in recs[:k]]
    dcg   = dcg_at_k(rels, k)
    ideal = [1]*min(len(gold_set), k)
    idcg  = dcg_at_k(ideal, k)
    return dcg / idcg if idcg>0 else 0.0

def evaluate(results, gold, k=4):
    P, R, T, RR, N = [], [], [], [], []
    for rid, ranked in results.items():
        if rid not in gold: 
            continue
        gs   = set(gold[rid])
        # ranked entries are (idx, score, job_id)
        recs = [job_id for _, _, job_id in ranked]

        P.append( precision_at_k(recs, gs, k) )
        R.append( recall_at_k(recs, gs, k)    )
        T.append( topk_accuracy(recs, gs, k)  )
        RR.append(reciprocal_rank(recs, gs)   )
        N.append( ndcg_at_k(recs, gs, k)      )

    return {
        f"Precision@{k}":   sum(P)/len(P),
        f"Recall@{k}":      sum(R)/len(R),
        f"Top-{k} Acc.":    sum(T)/len(T),
        "MRR":              sum(RR)/len(RR),
        f"NDCG@{k}":        sum(N)/len(N),
    }

In [16]:
metrics = evaluate(results_sbert, gold, k=4)
print("=== SBERT Evaluation on Gold Data ===")
for name, val in metrics.items():
    print(f"{name}: {val:.3f}")

=== SBERT Evaluation on Gold Data ===
Precision@4: 0.700
Recall@4: 0.700
Top-4 Acc.: 1.000
MRR: 0.900
NDCG@4: 0.733
